<a href="https://colab.research.google.com/github/sawamotokai/ASCII-art-generator/blob/main/ml/seq2seq-xy-numeric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import pandas as pd
from tqdm.notebook import trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [88]:
MAX_LENGTH = 50

class IMSIDataset(torch.utils.data.Dataset):
    def __init__(self, filepath):
      df = pd.read_parquet(filepath)
      df = filterDf(df)
      print(df.head)
      self.getStats(df)

      data = []
      cnt = 0
      for i in trange(len(df)):
        tmp = []
        row = df.loc[i]
        length = len(row['XBin'])
        for j in range(length):
          x = row['XBin'][j]
          y = row['YBin'][j]
          x, y = self.normalize(x, y)
          tmp.append([x, y])
        data.append(tmp)
      self.X = data
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        preds = self.X[idx]
        pol = self.y[idx]
        sample = { 'predictors' : preds, 'political' : pol }
        return sample

    def getStats(self, df):
      self.minX = np.inf
      self.minY = np.inf
      self.maxX = -np.inf
      self.maxY = -np.inf
      for i in trange(len(df)):
        row = df.loc[i]
        length = len(row['XBin'])
        for j in range(length):
          x = row['XBin'][j]
          y = row['YBin'][j]
          if x > self.maxX:
            self.maxX = x
          if y > self.maxY:
            self.maxY = y
          if x < self.minX:
            self.minX = x
          if y < self.minY:
            self.minY = y

    def normalize(self, x, y):
      x -= self.minX
      y -= self.minY
      return x / (self.maxX - self.minX), y / (self.maxY - self.minY)

    def denormalize(self, x, y):
      x *= (self.maxX - self.minX)
      y *= (self.maxY - self.minY)
      x += self.minX
      y += self.minY
      return x, y

def filterDf(df):
  indices = []
  for i in range(len(df)):
    if len(df.loc[i]['station']) <= MAX_LENGTH:
      indices.append(i)
  return df.loc[indices].reset_index().drop(columns=['index'])


def tensorFromCoords(coords):
  tensor = torch.tensor(coords, dtype=torch.float).to(device)
  return tensor


In [89]:

# convert data to torch.FloatTensor
transform = transforms.ToTensor()
# Create training and test dataloaders


file = "0.parquet"

dataset = IMSIDataset(file)
tr_len = int(len(dataset) * 0.9)

train_set = dataset.X[:tr_len]
valid_set = dataset.X[tr_len:]

# split = [tr_len, len(dataset)-tr_len]
# tr_data, val_data = torch.utils.data.random_split(dataset, split)
# num_workers = 0

<bound method NDFrame.head of                                                      bin  ...                                               YBin
0                         [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]  ...  [5039600, 5039600, 5038650, 5039750, 5039600, ...
1                                      [0.5, 0.25, 0.25]  ...                        [5012400, 5012050, 5012100]
2                                     [0.25, 0.75, 0.75]  ...                        [5082050, 5082000, 5082000]
3      [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75...  ...  [5023900, 5023800, 5024200, 5023900, 5023900, ...
4                                                 [0.25]  ...                                          [5044300]
...                                                  ...  ...                                                ...
71465                                             [0.75]  ...                                          [4853250]
71466                                  [0.5, 0.75, 0.75]  ...     

In [92]:
rand = random.choice(train_set)
print(dataset.denormalize(rand[0][0], rand[0][1]))

(606950.0, 5046750.0)


In [93]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.fc = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(2, hidden_size)

    def forward(self, input, hidden):
        # output = self.embedding(input).view(1, 1, -1)
        # print('before fc')
        # output = self.fc(input)
        # print(output.size())
        output = input.view(1,1,2)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
      
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.fc = nn.Linear(hidden_size, output_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden):
#         output = self.fc(input).view(1, 1, -1)
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)



class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        # self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.fc = nn.Linear(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # embedded = self.embedding(input).view(1, 1, -1)
        # print('begin decoder forward')
        # print(input.shape)
        embedded = self.fc(input)
        # print('after embedding')
        embedded = self.dropout(embedded)
        # print('after dropout')
        # print(embedded.size(), hidden[0].size())
        # print(torch.cat((embedded, hidden[0]), dim=1))

        attn_weights = F.softmax(self.attn(torch.cat((embedded, hidden[0]), dim=1)))
        # print('after softmax')
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        # print('after bmm')
        output = torch.cat((embedded[0], attn_applied[0].squeeze()), 0)
        output = self.attn_combine(output).unsqueeze(0)
        # print(output.size())
        output = F.relu(output)
        # print('after relu')
        output, hidden = self.gru(output.unsqueeze(0), hidden)
        # print('after gru')
        output = self.out(output[0])
        # output = F.log_softmax(self.out(output[0]), dim=1)
        # print('after softmax')
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [94]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import time
import math

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig('fig.png')

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [113]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # train decoder
    # print('input tensor size', input_tensor.size())
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        # print(encoder_output)
        encoder_outputs[ei] = encoder_output[0, 0]


    # train decoder 
    decoder_input = torch.zeros(1, 2).to(device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            # print("before using")
            # This is for attention
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # print(decoder_output)
            # print('decoder output')
            # decoder_output, decoder_hidden = decoder(
            #     decoder_input, decoder_hidden)
            # print(decoder_output)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di].unsqueeze(0)  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # print("before not using")
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # print(decoder_output)
            # decoder_output, decoder_hidden = decoder(
            # decoder_input, decoder_hidden)
            # topv, topi = decoder_output.topk(1)
            # print(decoder_output)
            # print('decoder ouput')
            # decoder_input = topi.squeeze().detach()  # detach from history as input
            decoder_input = decoder_output
            # print(decoder_input)
            loss += criterion(decoder_output, target_tensor[di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # training_pairs = [tensorsFromPair(random.choice(pairs))
                      # for i in range(n_iters)]

    X = dataset.X
    print("Before making pairs")
    training_pairs = [tensorFromCoords(random.choice(X)) for i in range(n_iters)]
    print("after making pairs")
    criterion = nn.MSELoss()

    for iter in trange(1, n_iters + 1):
        input_tensor = training_pairs[iter - 1]
        target_tensor = input_tensor.clone()
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [114]:
hidden_size = 256
encoder = EncoderRNN(2, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, 2).to(device)

trainIters(encoder, decoder, 7500, print_every=500)


Before making pairs
after making pairs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0m 14s (- 3m 17s) (500 6%) 0.0410
0m 28s (- 3m 3s) (1000 13%) 0.0229
0m 41s (- 2m 44s) (1500 20%) 0.0230
0m 54s (- 2m 31s) (2000 26%) 0.0253
1m 8s (- 2m 17s) (2500 33%) 0.0226
1m 22s (- 2m 4s) (3000 40%) 0.0230
1m 36s (- 1m 50s) (3500 46%) 0.0198
1m 49s (- 1m 36s) (4000 53%) 0.0180
2m 3s (- 1m 22s) (4500 60%) 0.0188
2m 16s (- 1m 8s) (5000 66%) 0.0195
2m 29s (- 0m 54s) (5500 73%) 0.0247
2m 42s (- 0m 40s) (6000 80%) 0.0239
2m 56s (- 0m 27s) (6500 86%) 0.0214
3m 10s (- 0m 13s) (7000 93%) 0.0208
3m 24s (- 0m 0s) (7500 100%) 0.0195



In [143]:
def evaluate(encoder, decoder, coords, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromCoords(coords)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]


        decoder_input = torch.zeros(1, 2).to(device)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(input_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            decoded_words.append(decoder_output.detach().tolist())
            decoder_input = decoder_output

        return decoded_words, decoder_attentions[:di + 1]


def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        coords = random.choice(valid_set)
        input = [dataset.denormalize(coord[0], coord[1]) for coord in coords]
        print('>', input)
        output, attentions = evaluate(encoder, decoder, coords)
        print('<', [dataset.denormalize(coord[0][0], coord[0][1]) for coord in output])
        print('')

In [145]:
evaluateRandomly(encoder, decoder, 5)

> [(531850.0, 5117850.0), (531850.0, 5117850.0), (531850.0, 5117850.0), (531750.0, 5117850.0), (531850.0, 5117800.0), (531850.0, 5117800.0), (531850.0, 5117850.0)]
< [(586708.9374214411, 5066800.345188379), (583795.2330887318, 5083797.880098224), (579207.8789591789, 5090504.540672898), (579551.6953468323, 5093814.66498971), (576629.9796581268, 5095337.487271428), (574884.6928328276, 5096469.850021601), (571116.8228924274, 5097696.854421496)]

> [(333300.0, 5184750.0), (333350.0, 5184700.0), (333300.0, 5184750.0), (333300.0, 5184700.0), (333200.0, 5184750.0), (333150.0, 5184750.0)]
< [(556764.7112131119, 5092221.53891623), (563787.0523899794, 5101621.170929074), (565504.4803708792, 5101975.274217129), (570248.4952747822, 5099804.011109471), (570139.0756577253, 5101221.67879045), (570344.5281744003, 5100189.000895619)]

> [(644650.0, 5099250.0), (644650.0, 5099300.0)]
< [(583280.5422306061, 5064571.574482322), (581539.3902987242, 5081690.24668932)]

> [(646400.0, 5091150.0), (646350.0, 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
